In [1]:
import mysql.connector
import prestodb.dbapi as presto
import pandas as pd
import numpy as np
import random
import datetime

# Create a Database to use

In [2]:
mysql = mysql.connector.connect(
    host='mysql',
    user='root',
    password='mysql'
)

In [3]:
database = "testing_db"
#
cur = mysql.cursor()
cur.execute(f"""

CREATE DATABASE IF NOT EXISTS {database}

""")
cur.fetchall()

[]

In [4]:
cur.execute(f"""

SHOW tables FROM {database}

""")
cur.fetchall()

[]

# Create Testing Tables

In [5]:
def modify_sql(sql_string):
    sql_string = sql_string.replace(" string", " varchar")
    return sql_string

In [6]:
conn = presto.Connection(
    host="presto", 
    port=8080, 
    user="demo"
)
cur = conn.cursor()
cur

In [7]:
cur.execute(modify_sql(f"""

CREATE TABLE IF NOT EXISTS target_db.testing_table_1(
    id string,
    color string,
    type int,
    number_of_coins bigint,
    measure decimal(19,3),
    start_date date
)

""").replace(" target_db.",f" mysql.{database}."))
cur.fetchall()

[[True]]

# Add in Table data

In [8]:
cur.execute(modify_sql(f"""

SHOW COLUMNS FROM target_db.testing_table_1

""").replace(" target_db.",f" mysql.{database}."))
cur.fetchall()

[['id', 'varchar', '', ''],
 ['color', 'varchar', '', ''],
 ['type', 'integer', '', ''],
 ['number_of_coins', 'bigint', '', ''],
 ['measure', 'decimal(19,3)', '', ''],
 ['start_date', 'date', '', '']]

In [9]:
def format_table_values(df):
    """format a dataframe into the syle of values typical for sql
    example: VALUES (1, 'a'), (2, 'b'), (3, 'c')
    """
    sending = []
    for i, row in df.iterrows():
        values = []
        for raw_value in row.tolist():
            if (type(raw_value) == str):
                if (raw_value.lower().startswith("date")):
                    values.append(str(raw_value))
                else: values.append(f"'{raw_value}'")
            else: values.append(str(raw_value))
        joined_values = ", ".join(values)
        sending.append(f"({joined_values})")
        
    if (len(sending) == 1):
        return sending[0]        
    else: return ", ".join(sending)

In [10]:
count = 10
example_df = pd.DataFrame(index=range(count))
example_df["id"] = random.choices(["123abc","456dfg","789hij"], k=count)
example_df["color"] = random.choices(["red","blue"],weights=[5, 1],k=count)
example_df["type"] = 7
example_df["number_of_coins"] = [(1 + x) * 100_000 for x in range(count)]
example_df["measure"] = [round(np.cos(m),2) for m in range(count)]
example_df["start_date"] = "date('"+pd.date_range(end='12/01/2021',periods=count,freq='D').astype(str)+"')"
example_df

,id,color,type,number_of_coins,measure,start_date
0,789hij,red,7,100000,1.00,date('2021-11-22')
1,789hij,red,7,200000,0.54,date('2021-11-23')
2,123abc,blue,7,300000,-0.42,date('2021-11-24')
3,123abc,red,7,400000,-0.99,date('2021-11-25')
4,123abc,red,7,500000,-0.65,date('2021-11-26')
5,456dfg,blue,7,600000,0.28,date('2021-11-27')
6,456dfg,blue,7,700000,0.96,date('2021-11-28')
7,789hij,blue,7,800000,0.75,date('2021-11-29')
8,123abc,blue,7,900000,-0.15,date('2021-11-30')
9,789hij,red,7,1000000,-0.91,date('2021-12-01')


In [11]:
cur.execute(f"""

INSERT INTO target_db.testing_table_1 VALUES {format_table_values(example_df)}

""".replace(" target_db.",f" mysql.{database}."))
cur.fetchall()

[[10]]

In [12]:
cur.execute(modify_sql(f"""

SELECT * FROM target_db.testing_table_1

""").replace(" target_db.",f" mysql.{database}."))
cur.fetchall()

[['789hij', 'red', 7, 100000, '1.000', '2021-11-22'],
 ['789hij', 'red', 7, 200000, '0.540', '2021-11-23'],
 ['123abc', 'blue', 7, 300000, '-0.420', '2021-11-24'],
 ['123abc', 'red', 7, 400000, '-0.990', '2021-11-25'],
 ['123abc', 'red', 7, 500000, '-0.650', '2021-11-26'],
 ['456dfg', 'blue', 7, 600000, '0.280', '2021-11-27'],
 ['456dfg', 'blue', 7, 700000, '0.960', '2021-11-28'],
 ['789hij', 'blue', 7, 800000, '0.750', '2021-11-29'],
 ['123abc', 'blue', 7, 900000, '-0.150', '2021-11-30'],
 ['789hij', 'red', 7, 1000000, '-0.910', '2021-12-01']]

In [13]:
conn.close()